# Session 3: Making your own classes - Methods, Events, Operators

We learned in our last session about how to make classes with properties, constructors, and fields.  We're going to continue adding features to our classes so that we can act on our classes and receive notifications about our classes.

## Methods

<div class="alert alert-block alert-info">
    Previously in Session 2, we wrote constructors to allow you to control the creation of classes.  These are _methods_ but this time we will expand on what we've previously learned
</div>

[Methods are defined in the official documentation](https://docs.microsoft.com/dotnet/csharp/programming-guide/classes-and-structs/methods) as a code block containing a series of statements that the program can execute.  We're going to expand on that definition to also state that a method can (but is not required) return values to its caller. We typically think of methods as an action that our class _performs_ or an _acting on_ the class.

A method has a **signature** that defines how you can interact with it.  The signature is followed immediately by curly braces **{ }** that wrap the code to be executed in the method and takes the following format:

_\[Access Modifier]*_ **Return Type** **Name**(_\[Parameters]_)  

Parameters are comma-delimited and defined similar to our variables with their type and name of the parameter:

`int myFirstParameter`

Let's take a look at some simple methods in a `Student` class:

In [1]:
class Student {
    
    public void Enroll() {
        // do enrollment steps
    }
    
    internal void LeaveClassEarly(DateTime departureTime) {
        // leave class code
    }
    
    public decimal CalculateGradeForClass(string className) {
        
        // grade calculation code
        
        // This method returns a value of type decimal 
        return 0.95m;
    }
    
}

var s = new Student();
display(s.CalculateGradeForClass("Quantum Physics"));

The below script needs to be able to find the current output cell; this is an easy method to get it.

0.95

Method access modifiers can be `public`, `protected`, `private`, `internal`, `protected internal`.  They can also be marked `static` to indicate that the method runs for all instances of the class and cannot access individual fields and properties of the class.  We'll see more about `static` and other modifiers like `override`, `virtual` and `abstract` in session 4.

### The Magic params Parameter and Optional Parameters

There is a special 'catch-all' `params` keyword you can use in your methods that will catch any arguments that are passed in to the method.  Let's update our student with the ability to calculate grade point average:

In [2]:
class Student {

    public decimal CalculateGradePointAverage(params string[] classes) {
        
        foreach(var c in classes) {
            display("Calculating for class " + c);
        }
        
        return 0.9m;
        
    }
    
}

var s = new Student();
display(s.CalculateGradePointAverage("Algebra", "History", "Computer Science"));

Calculating for class Algebra

Calculating for class History

Calculating for class Computer Science

0.9

Other arguments can be made optional by adding a default value to them:

In [3]:
class Student {
    
    public decimal CalculateGradePointAverage(short enrollmentYear = 2020, params string[] classes) {

        display("Calculating GPA for year " + enrollmentYear);
        
        foreach(var c in classes) {
            display("Calculating for class " + c);
        }
        
        return 0.9m;
        
    }
    
}

var s = new Student();
display(s.CalculateGradePointAverage(2019, "Algebra", "History", "Computer Science"));

Calculating GPA for year 2019

Calculating for class Algebra

Calculating for class History

Calculating for class Computer Science

0.9

### Out and Ref

Parameters passed into the method can **ALSO** take modifiers to set their interactions with the method.  The parameters can be passed both in and **OUT** as well as by value and by reference into the method.  This may seem a litle strange, so let's take a look at some samples.

We'll look at the `out` and `ref` keywords in this demo.  `out` specifies that a parameter is set as an output and `ref` indicates that a parameter is passed by reference.  If these keywords are used in a method's parameter signature, they must also be used when executing the method.

In [4]:
class Student {
    
    // Enroll and return a success indicator as well as the new StudentId
    public bool Enroll(short year, out string studentId) {
        
        // Enrollment code
        
        studentId = "1234567";
        return true;
        
    }

    public void DoSomething(ref int myValue) {
        myValue++;
    }
    
}

var s= new Student();
string id = "";
short year = 2020;

display(s.Enroll(year, out id));
display("student id: " + id);
display("Year: " + year);

True

student id: 1234567

Year: 2020

The id is set in our method and MUST be set before the method ends to be returned properly.  Let's look at the `DoSomething` method that receives a value **ByReference** as indicated by the `ref` keyword.  Check out the behavior of `myValue` and tinker with adding and removing the `ref` keyword to see its behavior.

In [5]:
int myValue = 10;
s.DoSomething(ref myValue);
display(myValue);

11

The `myValue` integer is a **value type** and is passed as a reference into the `DoSomething` method where it is modified.  The reference to the `myValue` variable is updated inside the method, and its value reflects that update.

Reference types, those types we create with the `class` keyword like `Student` are **ALWAYS** passed by reference into methods. 

## Delegates

Now that we know what a method is and how to interact with them, sometimes we want to pass a pointer to that method around our program.  This pointer to the method is called a **Delegate** and allows us to call the method from another location.  Delegates are defined with the method signature that they need to match in order to reference that method.  The [official documentation on delegates](https://docs.microsoft.com/dotnet/csharp/programming-guide/delegates/) has more on defining a delegate.

Let's take a look at passing a delegate for a pointer into another method.

In [6]:

class Student {
    
    public delegate int CalculateHandler(int myArg1, int myArg2);
    
    public int Calculate(int arg1, int arg2, CalculateHandler handler) {
        
        var output = handler(arg1, arg2);
        return output;
        
    }
    
    public int Add(int arg1, int arg2) {
        var output = arg1 + arg2;
        display("Added: " + output);
        return output;
    }
    
    public int Subtract(int arg1, int arg2) {
        var output = arg1 - arg2;
        display("Subtracted: " + output);
        return output;
    }
    
}

var s = new Student();

// instantiating a delegate is used with a NEW keyword wrapping the name of the method to be assigned
var calcHandler = new Student.CalculateHandler(s.Add);
s.Calculate(10, 5, calcHandler);

Added: 15

We can also assign **anonymous methods** to a delegate to be passed around.  An anonymous method is defined with some paraenthesis enclosing the parameter list and an expression body indicated with the fat-arrow `=>` notation.  

Consider this code to work with our Student CalculateHandler:

In [7]:
var s = new Student();

var multiply = new Student.CalculateHandler((int arg1, int arg2) => arg1 * arg2);
display(s.Calculate(10, 5, multiply));

50

We can also directly cast a delegate when the type is specifed, allowing for implicit conversion to the delegate type.  This makes our code a little more terse, but the intent is still clear:


In [8]:
var s = new Student();

s.Calculate(10, 5, s.Add);

Added: 15

### Multicast delegates

Defined delegate types can be **multicast** allowing them to point to multiple methods to be called.  This sounds a LITTLE weird, but it means that we can stack executions and pass that entire stack into another method.  Let's take a look at that `Student` and `CalculateHandler` from our previous example again.

In [9]:
var s = new Student();

// Define an initial delegate variable
var calculation = new Student.CalculateHandler(s.Add) + new Student.CalculateHandler(s.Subtract);

/*
calculation += new Student.CalculateHandler((int arg1, int arg2) => {
    var outValue = arg1 * arg2;
    display("Multiplied: " + outValue);
    return outValue;
});
*/

s.Calculate(10, 5, calculation);

Added: 15

Subtracted: 5

Delegates can be combined with `+` and `-` operators to stack them and remove them from the stack to be executed.  As we'll see with **Events** in the next section, adding and removing references to delegates is very important.

## Events

[Events](https://docs.microsoft.com/dotnet/csharp/programming-guide/events/) allow us to notify when something has happened inside of our class.  Events build on the concept of delegates as they reference another method that should be called when the event is **raised**.  We define an event with access modifiers and by .NET standard practice, two arguments:  the sender and a class that contains any arguments about the event being raised.  The return type from an event is a delegate of type `EventHandler` that defines these two arguments.  

Let's look at a simple example:

In [10]:
class EnrolledEventArgs : EventArgs {
    public short YearEnrolled {get; set;}
}

class Student {
    
    
    public delegate void EnrolledEventHandler(object sender, EnrolledEventArgs args);
    public event EnrolledEventHandler Enrolled;
    
    public void Enroll() {
        
        // do something
        Enrolled(this, new EnrolledEventArgs {YearEnrolled = 2020});
        
    }
    
}

var s = new Student();
s.Enrolled += (sender, args) => display("I'm now enrolled for the year " + args.YearEnrolled);
// s.Enroll();